# understanding the features

this analysis is build uses the joined dataset.


In [1]:
import os
import sys
module_path2 = os.path.abspath(os.path.join('..'))
x2 = module_path2+"/email_lead_scoring"
if x2 not in sys.path:
    sys.path.append(x2)
print(sys.path)

['/Users/jp/GitHub/ds-python-2/03_data_pipelines', '/Users/jp/miniconda3/envs/python_ds2_3.9/lib/python39.zip', '/Users/jp/miniconda3/envs/python_ds2_3.9/lib/python3.9', '/Users/jp/miniconda3/envs/python_ds2_3.9/lib/python3.9/lib-dynload', '', '/Users/jp/miniconda3/envs/python_ds2_3.9/lib/python3.9/site-packages', '/Users/jp/GitHub/ds-python-2/email_lead_scoring']


In [2]:
os.getcwd()
x2
#os.path.abspath(os.path.join('..'))

'/Users/jp/GitHub/ds-python-2/email_lead_scoring'

In [3]:

import pandas as pd
import numpy as np
import plotly.express as px
# EDA 
import re
import janitor as jn
import sweetviz as sv
# Email Lead Scoring
from read_datas import join_crm_datas
from eda import sales_by_category



/Users/jp/miniconda3/envs/python_ds2_3.9/lib/python3.9/site-packages/janitor/utils.py:84: FutureWarning: PandasArray has been renamed NumpyExtensionArray. Use that instead. This alias will be removed in a future version.
  @_expand_grid.register(pd.arrays.PandasArray)


In [4]:
leads_df = join_crm_datas('no')

### cardinality
high cardinality means that data has many unique values, for example, a column with a person's name has high cardinality because there are many different names. cardinality

In [5]:
leads_df.T

,0,1,2,3,4,5,6,7,8,9,...,19909,19910,19911,19912,19913,19914,19915,19916,19917,19918
mailchimp_id,3,4,8,10,11,14,18,19,20,22,...,33395,33396,33400,33402,33403,33405,33406,33407,33408,33409
user_full_name,Garrick Langworth,Cordell Dickens,Inga Dach,Ferdinand Bergstrom,Justen Simonis,Marion Corwin,Dr Sharif Kunde,Marlena Reichert d.d.s.,Mr. Brandan Oberbrunner,Keesha Schowalter,...,Royce Franecki-Stoltenberg,Alvis Spencer-Jones,Treyton Kirlin,Miss. Margeret Labadie,Sybil Hackett,Ms. Felicity Moore MD,Shirley Rowe,Jarrett Walker-Carroll,Tanja Herzog,Farrell Murphy
user_email,garrick.langworth@gmail.com,cordell.dickens@gmail.com,inga.dach@gmail.com,ferdinand.bergstrom@gmail.com,justen.simonis@gmail.com,marion.corwin@hotmail.com,dr.sharif.kunde@gmail.com,marlena.reichert.dds@gmail.com,mr.brandan.oberbrunner@gmail.com,keesha.schowalter@hotmail.com,...,royce.franeckistoltenberg@gmail.com,alvis.spencerjones@gmail.com,treyton.kirlin@gmail.com,miss.margeret.labadie@gmail.com,sybil.hackett@gmail.com,ms.felicity.moore.md@gmail.com,shirley.rowe@gmail.com,jarrett.walkercarroll@gmail.com,tanja.herzog@gmail.com,farrell.murphy@gmail.com
member_rating,2,4,2,2,2,2,2,2,4,2,...,2,2,2,2,2,2,1,2,2,2
optin_time,2019-05-22,2018-11-19,2018-11-19,2020-03-20,2020-04-14,2019-11-04,2020-01-22,2020-12-06,2019-07-10,2019-04-10,...,2020-10-27,2019-10-10,2020-06-08,2020-07-22,2019-11-25,2018-11-18,2019-03-12,2019-09-09,2019-10-24,2020-06-19
country_code,in,it,<NA>,co,<NA>,mx,<NA>,in,us,sg,...,fr,<NA>,it,<NA>,<NA>,se,br,in,<NA>,ae
tag_count,6.0,0.0,0.0,3.0,0.0,0.0,2.0,0.0,4.0,2.0,...,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,2.0,0.0
made_purchase,1,1,1,1,0,0,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0


most of the sales comes from US and India, when sales is set arbirary over 5, the country list goes down from 183 to 21. Generally its good idea to group there countries to 'other' group since they add little predictive value.

In [6]:
sales_by_cat = sales_by_category(leads_df, 'country_code', sort_by='sales').reset_index()

In [7]:
sales_by_cat_ab5 = sales_by_cat.query('sales > 5')
sales_by_cat_ab5

,country_code,sales,sales_ratio_per_country,ratio_of_total_sales,cumulative_sum
0,us,406,0.107209,0.464531,0.464531
1,in,76,0.027890,0.086957,0.551487
2,au,37,0.121311,0.042334,0.593822
3,uk,32,0.062992,0.036613,0.630435
4,br,25,0.068306,0.028604,0.659039
5,ca,24,0.061538,0.027460,0.686499
6,de,22,0.078571,0.025172,0.711670
7,fr,13,0.045614,0.014874,0.726545
8,es,12,0.043165,0.013730,0.740275
9,mx,11,0.061111,0.012586,0.752860


In [8]:

fig = px.bar(sales_by_cat, x='country_code', y='sales', color='sales', title='Sales by Country')
fig.show()

In [9]:
above5_px = px.bar(sales_by_cat_ab5, x='country_code', y='sales', color='sales', title='Sales by Country')
above5_px.show()


split by member group

In [10]:
sales_by_category(leads_df, 'member_rating', sort_by='sales').reset_index().sort_values('member_rating')

,member_rating,sales,sales_ratio_per_country,ratio_of_total_sales,cumulative_sum
4,1,2,0.009804,0.002079,1.000000
0,2,435,0.028988,0.452183,0.452183
3,3,80,0.077071,0.083160,0.997921
2,4,110,0.082335,0.114345,0.914761
1,5,335,0.143469,0.348233,0.800416


In [13]:
sales_by_category(leads_df, 'country_code')

,sales,sales_ratio_per_country,ratio_of_total_sales,cumulative_sum
country_code,,,,
us,406,0.107209,0.464531,0.464531
in,76,0.027890,0.086957,0.551487
au,37,0.121311,0.042334,0.593822
uk,32,0.062992,0.036613,0.630435
br,25,0.068306,0.028604,0.659039
...,...,...,...,...
dm,0,0.000000,0.000000,1.000000
cy,0,0.000000,0.000000,1.000000
cw,0,0.000000,0.000000,1.000000
